In [1]:
run src/midi_processing.py

In [2]:
def one_hot_song(midi):
    song = converter.parse(midi)
    seconds_map = song.flat.getElementsByClass(['Note', 'Rest', 'Chord']).secondsMap
    one_hot_list = []
    for note in seconds_map:
        note_length = round(note['durationSeconds'],3)
        note_type = note['element']
        if note_type.isRest:
            if note_length > 3.5: #check if time shift is greater than three and a half seconds 
                note_length = 3.0
            while note_length > 0.0:
                if note_length <= 1.0: #if note length is greater than the maximum encodable time shift
                    one_hot_list.append(one_hot_time(note_length))
                    note_length = 0.0
                elif note_length > 1.0:
                    one_hot_list.append(one_hot_time(1.0))
                    note_length -= 1.0
                
                    
        elif note_type.isChord: #check if element is a chord
            vel = get_vel_from_chord(note_type)
            note_lst = get_notes_from_chord(note_type) #if element is a chord, get all notes
            for note in note_lst: #one hot each note-on in the chord
                one_hot_list.append(one_hot_note_on(note)) 
            one_hot_list.append(one_hot_vel(vel)) #one hot velocity
            while note_length > 0.0:
                if note_length <= 1.0: #if note length is greater than the maximum encodable time shift
                    one_hot_list.append(one_hot_time(note_length))
                    note_length = 0.0
                elif note_length > 1.0:
                    one_hot_list.append(one_hot_time(1.0))
                    note_length -= 1.0            
            for note in note_lst:
                one_hot_list.append(one_hot_note_off(note)) #turn off each note in chord

        else:
            vel = note_type.volume.velocity #get velocity from note
            pitch = get_pitch_from_note(note['element'])
            one_hot_list.append(one_hot_note_on(pitch)) #one hot note-on
            one_hot_list.append(one_hot_vel(vel))
            if note_length <= 1.0: #if note length is greater than the maximum encodable time shift
                    one_hot_list.append(one_hot_time(note_length))
                    note_length = 0.0
            elif note_length > 1.0:
                    one_hot_list.append(one_hot_time(1.0))
                    note_length -= 1.0 
            one_hot_list.append(one_hot_note_off(pitch))  
    return one_hot_list